Kod pozwala na analizę kinemtyczną wygenerowanych zdarzeń. Do jego użycia potrzebne są pliki .csv otrzymane z użyciem skryptu Eksport_danych_z_root.ipynb.

In [ ]:
import csv
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np


In [ ]:
# Wczytanie danych
def loading_data_restructured(file):
    with open(file, 'r', encoding='utf-8') as f:
        all_events = []
        multiplicities = []

        reader = csv.reader(f)
        header = next(reader)  # Pomijamy nagłówek

        for row in reader:
            event_num = int(row[0])
            interaction_type = row[1]
            particles = []
            for i in range(3, len(row), 5):
                try:
                    pid = int(row[i])
                    px = float(row[i+1])
                    py = float(row[i+2])
                    pz = float(row[i+3])
                    E  = float(row[i+4])
                    p = np.array([px, py, pz])
                    M = np.sqrt(max(E**2 - np.dot(p, p), 0))

                    particles.append([pid, px, py, pz, E, M])
                except (ValueError, IndexError):
                    continue  # pomiń cząstki niepełne lub błędne

            if not particles:
                continue  # pomiń zdarzenia bez żadnej cząstki

            multiplicities.append(len(particles))

            total_px = sum(p[1] for p in particles)
            total_py = sum(p[2] for p in particles)
            total_pz = sum(p[3] for p in particles)
            total_E  = sum(p[4] for p in particles)
            total_p  = np.array([total_px, total_py, total_pz])
            total_M  = np.sqrt(max(total_E**2 - np.dot(total_p, total_p), 0))

            event = [event_num, interaction_type, total_px, total_py, total_pz, total_E, total_M]
            for p in particles:
                event.extend(p)

            all_events.append(event)

    return all_events, multiplicities


Definiujemy funkcje, które zwracają interesujące nas zmienne kinematyczne.

In [ ]:

# Funkcja zwraca całkowity pęd poprzeczny
def extract_transverse_momentum(events):
    pt_list = []
    for event in events:
        px, py = event[2], event[3]
        pt = np.sqrt(px**2 + py**2)
        pt_list.append(pt)
    return pt_list

# Funkcja zwraca parametry kinematyczne zapisane w "event"
def extract_event_kinematics(event):
    total_px = event[2]
    total_py = event[3]
    total_pz = event[4]
    total_E  = event[5]
    total_M  = event[6]
    return total_E, total_M, total_px, total_py, total_pz

def extract_kinematics_lists(events):
    Es, Ms, Pxs, Pys, Pzs = [], [], [], [], []
    for event in events:
        E, M, Px, Py, Pz = extract_event_kinematics(event)
        Es.append(E)
        Ms.append(M) 
        Pxs.append(Px)
        Pys.append(Py)
        Pzs.append(Pz)
    return Es, Ms, Pxs, Pys, Pzs


# Funkcja sprawdza czy zdarzenie ma końcowa konfigurację cząstek 3 piony + proton
def filter_proton_3pion(events):
    filtered = []
    for event in events:
        pids = [int(event[i]) for i in range(7, len(event), 6)]
        if pids.count(2212) == 1 and (pids.count(211) + pids.count(-211)) == 3 and len(pids) == 4:
            filtered.append(event)
    return filtered

# Funkcja zwraca dodatkowe zmienne kinematyczne
def extract_additional_kinematics(events):
    modP, angleZ, asym_x, asym_y, asym_z = [], [], [], [], []

    for event in events:
        Px, Py, Pz = event[2], event[3], event[4]
        p = np.array([Px, Py, Pz])
        norm_p = np.linalg.norm(p)

        modP.append(norm_p)

        if norm_p != 0:
            angle = np.arccos(Pz / norm_p)
            angleZ.append(np.degrees(angle))  # stopnie
            asym_x.append(abs(Px) / norm_p)
            asym_y.append(abs(Py) / norm_p)
            asym_z.append(abs(Pz) / norm_p)
        else:
            angleZ.append(0)
            asym_x.append(0)
            asym_y.append(0)
            asym_z.append(0)

    return modP, angleZ, asym_x, asym_y, asym_z

# Funkcja licząca sferyczność
def compute_sphericity(event):
    particles = []
    for i in range(7, len(event), 6):
        try:
            px = event[i+1]
            py = event[i+2]
            pz = event[i+3]
            particles.append([px, py, pz])
        except IndexError:
            continue

    if len(particles) < 2:
        return 0

    S = np.zeros((3, 3)) #Tworzymy macierz 3x3
    denominator = 0
    # Liczymy macierz sferyczności zgodnie ze wzorem
    for p in particles:
        p_vec = np.array(p)
        denominator += np.dot(p_vec, p_vec) 
        for i in range(3):
            for j in range(3):
                S[i][j] += p_vec[i] * p_vec[j]

    if denominator == 0:
        return 0

    S /= denominator
    # Liczymy wartości własne macierzy i sortujemy je od największej do najmniejszej
    eigenvalues = np.linalg.eigvalsh(S)
    eigenvalues = sorted(eigenvalues, reverse=True)
    # Liczymy sferyczność
    sphericity = 1.5 * (eigenvalues[1] + eigenvalues[2])
    return sphericity

def extract_sphericities(events):
    return [compute_sphericity(event) for event in events]

Wczytujemy dane oraz ekstraktujemy z nich zmienne kinematyczne

In [ ]:

files = ['nuTau_data_CCQES.csv', 'nuTau_data_NC_6mil.csv', 'nuTau_data_NC_6mil_2.csv', 'nuTau_data_NC_3mil.csv']

all_events = []
all_multiplicities = []

for f in files:
    events, mult = loading_data_restructured(f)
    all_events.extend(events)
    all_multiplicities.extend(mult)

print(f"Załadowano łącznie {len(all_events)} zdarzeń.")

# Filtrowanie danych
events_1p3pi = filter_proton_3pion(all_events)
events_NC = [e for e in events_1p3pi if e[1] == 'NC']
events_CC = [e for e in events_1p3pi if e[1] == 'CC']

# Ekstrakcja danych
E_NC, M_NC, Px_NC, Py_NC, Pz_NC = extract_kinematics_lists(events_NC)
E_CC, M_CC, Px_CC, Py_CC, Pz_CC = extract_kinematics_lists(events_CC)
pt_NC = extract_transverse_momentum(events_NC)
pt_CC = extract_transverse_momentum(events_CC)

# Moduł pędu, kąt z osią Z, asymetria
modP_NC, angleZ_NC, asym_x_NC, asym_y_NC, asym_z_NC = extract_additional_kinematics(events_NC)
modP_CC, angleZ_CC, asym_x_CC, asym_y_CC, asym_z_CC = extract_additional_kinematics(events_CC)
# Sferyczność
sphericity_NC = []
sphericity_CC = []
for event in events_NC:
    s, a, p = compute_sphericity(event)
    sphericity_NC.append(s)

for event in events_CC:
    s, a, p = compute_sphericity(event)
    sphericity_CC.append(s)

Definiujemy funkcję rysującą rozkłady zmiennych kinematycznych

In [ ]:
def plot_all_distributions(
    Px_NC, Py_NC, Pz_NC, E_NC, M_NC, pt_NC, spher_NC, theta_NC,
    Px_CC, Py_CC, Pz_CC, E_CC, M_CC, pt_CC, spher_CC, theta_CC,
):
    fig, axs = plt.subplots(2, 4, figsize=(24, 10))

    # Lista parametrów
    params = [
        (Px_NC, Px_CC, "Px [GeV]", "Rozkład Px"),
        (Py_NC, Py_CC, "Py [GeV]", "Rozkład Py"),
        (Pz_NC, Pz_CC, "Pz [GeV]", "Rozkład Pz"),
        (E_NC, E_CC, "E [GeV]", "Rozkład energii całkowitej"),
        (M_NC, M_CC, "Masa [GeV]", "Rozkład masy niezmienniczej"),
        (pt_NC, pt_CC, "pT [GeV]", "Rozkład pędu poprzecznego"),
        (spher_NC, spher_CC, "Sferyczność", "Rozkład sferyczności"),
        (theta_NC, theta_CC, "Kąt do osi z [°]", "Rozkład kąta pędu do osi z")
    ]

    for i, (data_NC, data_CC, xlabel, title) in enumerate(params):
        combined_data = np.concatenate([data_NC, data_CC])
        bins = np.histogram_bin_edges(combined_data, bins=50)

        ax = axs[i // 4, i % 4]
        ax.hist(data_NC, bins=bins, alpha=0.6, label="NC", color='tab:blue', density=True, edgecolor='black')
        ax.hist(data_CC, bins=bins, alpha=0.6, label="CC", color='tab:orange', density=True, edgecolor='black')

        ax.set_xlabel(xlabel, fontsize=14)
        ax.set_ylabel("Znormalizowana częstość", fontsize=14)
        ax.set_title(title, fontsize=16)
        ax.tick_params(axis='both', labelsize=12)
        ax.legend(fontsize=12)
        ax.grid(True, linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.show()

In [ ]:
plot_all_distributions(
    Px_NC, Py_NC, Pz_NC, E_NC, M_NC, pt_NC, sphericity_NC, angleZ_NC,
    Px_CC, Py_CC, Pz_CC, E_CC, M_CC, pt_CC, sphericity_CC, angleZ_CC,
)

Dalsza część kodu pozwala na uzyskanie wykresu częstości obecności cząstek w zdarzeniach o krotności równej 4.

In [ ]:
# Lista kodów PDG cząstek oraz odpowiadające im nazwy
particle_names = {
    2212: 'proton',
    211: 'pi+',
    -211: 'pi-',
    321: 'K+',
    -321: 'K-',
    13: 'mu-',
    -13: 'mu+',
    11: 'e-',
    -11: 'e+',
    22: 'photon',
    3112: 'Sigma-',
    3222: 'Sigma+',
    4122: 'Lambda_c+',
    -2212: r'$\bar{p}$',
    -2112: r'$\bar{n}$',
    411: 'D+',
    431: 'D+s',
    111: 'pi 0'
}

# Funkcja rysująca histogramy częstości występywania cząstek
def plot_particle_frequencies(events):

    counter_NC = Counter()
    counter_CC = Counter()

    total_NC = 0
    total_CC = 0

    for event in events:
        interaction_type = event[1]
        pids = [int(event[i]) for i in range(7, len(event), 6)]
        if not pids:
            continue

        if interaction_type == 'NC':
            counter_NC.update(pids)
            total_NC += 1
        elif interaction_type == 'CC':
            counter_CC.update(pids)
            total_CC += 1

    # Wszystkie PIDy do wykresu
    all_pids = sorted(set(counter_NC.keys()) | set(counter_CC.keys()))
    labels = [particle_names.get(pid, str(pid)) for pid in all_pids]

    values_NC = [100 * counter_NC.get(pid, 0) / total_NC for pid in all_pids]
    values_CC = [100 * counter_CC.get(pid, 0) / total_CC for pid in all_pids]

    x = np.arange(len(all_pids))
    width = 0.35

    fig, ax = plt.subplots(figsize=(12, 7))
    rects1 = ax.bar(x - width/2, values_NC, width, label='NC', color='skyblue')
    rects2 = ax.bar(x + width/2, values_CC, width, label='CC', color='lightcoral')

    ax.set_ylabel('Procent zdarzeń [%]')
    ax.set_title('Występowanie cząstek w zdarzeniach NC i CC')
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=45)
    ax.legend()
    ax.grid(axis='y')

    # Dodanie wartości nad słupkami
    for rect in rects1 + rects2:
        height = rect.get_height()
        ax.annotate(f'{height:.1f}%',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=8)

    plt.tight_layout()
    plt.show()

In [ ]:

plot_particle_frequencies(events)